In [2]:
import os

def baixar_arquivo(url, local_filename=None):
    """Baixa e salva o arquivo de uma url."""
    # Se o nome do arquivo não foi passado como parâmetro, extrair da url.
    if local_filename is None:
        local_filename = os.path.join("./tarifaSocial", url.split('/')[-1])

    #verifica se arquivo já foi baixado
    if os.path.exists(local_filename):
        print(f'Arquivo {local_filename} já baixado!')
        return None
    else:
        print('Solicitando arquivo do servidor.')
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(local_filename, 'wb') as f:
            print(f"Baixando arquivo da url: {url}")
            for chunk in response.iter_content(chunk_size=8192): 
                f.write(chunk)
            return "Arquivo baixado!"
     

In [3]:
def read_page(url):
    print(f'Fazendo requisição GET para o link: {url}')
    response = requests.get(url)
    response.raise_for_status() #Levanta uma exceção caso haja erro na requisição.
    
    if response.status_code == 200:
        content = response.text
        return content

In [4]:
import requests
from bs4 import BeautifulSoup
import time

TIME_TO_SLEEP = 30

url = 'https://dadosabertos.aneel.gov.br/dataset/beneficiarios-da-cde'

content = read_page(url)
soup = BeautifulSoup(content, 'html.parser')

Fazendo requisição GET para o link: https://dadosabertos.aneel.gov.br/dataset/beneficiarios-da-cde


In [5]:
#https://dadosabertos.aneel.gov.br/dataset/beneficiarios-da-cde

In [6]:
 section = soup.find('ul', class_='resource-list')
 links=section.find_all('a', class_='resource-url-analytics')

In [7]:
len(links)

70

In [8]:
tarifaSocial=[]

for item in links:
    link = item['href']
    term_link = link[len(link)-3:]
    if term_link == 'zip':
        tarifaSocial.append(link)

In [9]:
tarifaSocial[-10:]

['https://dadosabertos.aneel.gov.br/dataset/a7191647-b187-4893-b20a-8954d57ff89c/resource/75e1c189-010c-4186-a2b6-2d889ada6927/download/cde-beneficiarios-01nov2022.zip',
 'https://dadosabertos.aneel.gov.br/dataset/a7191647-b187-4893-b20a-8954d57ff89c/resource/20a283ca-7c3e-42d5-8f7b-2f2fde25c980/download/cde-beneficiarios-01dec2022.zip',
 'https://dadosabertos.aneel.gov.br/dataset/a7191647-b187-4893-b20a-8954d57ff89c/resource/bd8004f6-2b36-45a9-b10f-cb05e7787451/download/cde-beneficiarios-01jan2023.csv.zip',
 'https://dadosabertos.aneel.gov.br/dataset/a7191647-b187-4893-b20a-8954d57ff89c/resource/8e2c96fb-78a6-478c-97aa-707ad33db5b3/download/cde-beneficiarios-01feb2023.csv.zip',
 'https://dadosabertos.aneel.gov.br/dataset/a7191647-b187-4893-b20a-8954d57ff89c/resource/456a0e7c-f4d3-401e-8cec-b4299253b4ee/download/cde-beneficiarios-01mar2023.csv.zip',
 'https://dadosabertos.aneel.gov.br/dataset/a7191647-b187-4893-b20a-8954d57ff89c/resource/4cefdddd-675a-4b6b-b188-466242cb0aca/download/cd

In [ ]:
import pandas as pd

In [10]:
for item in tarifaSocial[-9:-8]:

    if baixar_arquivo(item):
        print('Esperando 120s para solicitar próximo download...')
        time.sleep(TIME_TO_SLEEP)
    else:
        print("Link para download não encontrado.")  
     
print('fim')

Arquivo ./tarifaSocial\cde-beneficiarios-01dec2022.zip já baixado!
Link para download não encontrado.
fim


In [11]:
import os
import zipfile

#caminho = os.getcwd()
caminho = 'tarifaSocial'
arquivos = os.listdir(caminho)



In [12]:

arquivos



['.ipynb_checkpoints',
 'cde-beneficiarios-01apr2022.zip',
 'cde-beneficiarios-01aug2023.csv.zip',
 'cde-beneficiarios-01DEC2022.csv',
 'cde-beneficiarios-01dec2022.zip',
 'cde-beneficiarios-01feb2022.zip',
 'cde-beneficiarios-01jan2022.zip',
 'cde-beneficiarios-01jul2023.csv.zip',
 'cde-beneficiarios-01mar2022.zip',
 'cde-beneficiarios-01may2022.zip']

In [14]:
for arquivo in arquivos[3:4]:
    if arquivo.endswith('.zip'):
        caminho_completo = os.path.join(caminho, arquivo)

        with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
            zip_ref.extractall(caminho)

In [15]:
#arquivos=[]
#for file in os.listdir(caminho):
#    if file.endswith(".xlsx"):
#        print(f'Carregando arquivo {file} ...')
#        arquivos.append(pd.read_excel(os.path.join(caminho,file), sheet_name="Base tratada"))

In [16]:
#arquivos.head()

In [17]:
import glob
os.chdir(caminho)
arquivos2 = glob.glob('*.csv')


In [73]:
arquivos2

['cde-beneficiarios-01DEC2022.csv']

In [74]:
import pandas as pd
tabelas=[]

for x in arquivos2:
    print(f'Carregando arquivo {x} ...')
    tabelas.append(pd.read_csv(x,sep=',',encoding='ISO-8859-1'))

print('fim')
   
    

Carregando arquivo cde-beneficiarios-01DEC2022.csv ...


C:\Users\aline\AppData\Local\Temp\ipykernel_5768\1762207804.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tabelas.append(pd.read_csv(x,sep=',',encoding='ISO-8859-1'))


fim


In [75]:
len(tabelas)

1

In [76]:
tabela_final=pd.concat(tabelas)

In [77]:
tabela_final['VlrSubsidio']=tabela_final['VlrSubsidio'].str.replace('.','')

In [78]:
tabela_final['VlrSubsidio']=tabela_final['VlrSubsidio'].str.replace(',','.')

In [79]:
tabela_final['VlrSubsidio']=tabela_final['VlrSubsidio'].astype(float)

In [80]:
tabela_final2=tabela_final[(tabela_final['DscTipoSubsidio']=='SubsBaixaRenda')]

In [81]:
tabela_final2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15621465 entries, 1 to 20582674
Data columns (total 14 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   DatGeracaoConjuntoDados  object 
 1   AnmReferencia            object 
 2   SigAgente                object 
 3   NumCNPJDistribuidora     int64  
 4   CodIbgeMunicipio         int64  
 5   IdcTipoFaturamento       int64  
 6   IdcClasseConsumidor      int64  
 7   IdcSubclasse             float64
 8   IdcSubgrupoTarifario     int64  
 9   IdcTipoCompraEnergia     int64  
 10  NomCliente               object 
 11  NumCPFCNPJCliente        object 
 12  DscTipoSubsidio          object 
 13  VlrSubsidio              float64
dtypes: float64(2), int64(6), object(6)
memory usage: 1.7+ GB


In [92]:
tabela_final2.columns

Index(['DatGeracaoConjuntoDados', 'AnmReferencia', 'SigAgente',
       'NumCNPJDistribuidora', 'CodIbgeMunicipio', 'IdcTipoFaturamento',
       'IdcClasseConsumidor', 'IdcSubclasse', 'IdcSubgrupoTarifario',
       'IdcTipoCompraEnergia', 'NomCliente', 'NumCPFCNPJCliente',
       'DscTipoSubsidio', 'VlrSubsidio'],
      dtype='object')

In [93]:
tabela_final2[:5]

,DatGeracaoConjuntoDados,AnmReferencia,SigAgente,NumCNPJDistribuidora,CodIbgeMunicipio,IdcTipoFaturamento,IdcClasseConsumidor,IdcSubclasse,IdcSubgrupoTarifario,IdcTipoCompraEnergia,NomCliente,NumCPFCNPJCliente,DscTipoSubsidio,VlrSubsidio
1,18/08/2023,12/2022,CERAL ARARUAMA,28610236000169,3304300,1,1,3.2,11,1,ELIAS DOS SANTOS,***.572.237-**,SubsBaixaRenda,70.88
3,18/08/2023,12/2022,CERAL ARARUAMA,28610236000169,3304300,1,1,3.2,11,1,NILSETE DOS SANTOS LIMA,***.364.757-**,SubsBaixaRenda,32.76
4,18/08/2023,12/2022,CERAL ARARUAMA,28610236000169,3304300,1,1,3.2,11,1,LUCIANO LEONARDO DA SILVA,***.502.907-**,SubsBaixaRenda,52.30
5,18/08/2023,12/2022,CERAL ARARUAMA,28610236000169,3304300,1,1,3.2,11,1,ANGELA MARIA DE SOUZA,***.787.147-**,SubsBaixaRenda,30.85
18,18/08/2023,12/2022,CERAL ARARUAMA,28610236000169,3304300,1,1,3.2,11,1,NELMA ALVES MARINS,***.798.297-**,SubsBaixaRenda,62.67


In [94]:
BaixaRenda_Num=tabela_final2.groupby(['AnmReferencia', 'CodIbgeMunicipio', 'IdcTipoFaturamento',
       'IdcClasseConsumidor', 'IdcSubclasse', 'IdcSubgrupoTarifario',
       'IdcTipoCompraEnergia','DscTipoSubsidio'])['VlrSubsidio'].agg(['sum','count'])



In [95]:
baixaRenda=BaixaRenda_Num.reset_index()

In [96]:
baixaRenda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20068 entries, 0 to 20067
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AnmReferencia         20068 non-null  object 
 1   CodIbgeMunicipio      20068 non-null  int64  
 2   IdcTipoFaturamento    20068 non-null  int64  
 3   IdcClasseConsumidor   20068 non-null  int64  
 4   IdcSubclasse          20068 non-null  float64
 5   IdcSubgrupoTarifario  20068 non-null  int64  
 6   IdcTipoCompraEnergia  20068 non-null  int64  
 7   DscTipoSubsidio       20068 non-null  object 
 8   sum                   20068 non-null  float64
 9   count                 20068 non-null  int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 1.5+ MB


In [97]:
BaixaRenda2=tabela_final2[(tabela_final2['IdcTipoFaturamento']==1)]


In [98]:
BaixaRenda2.tail()

,DatGeracaoConjuntoDados,AnmReferencia,SigAgente,NumCNPJDistribuidora,CodIbgeMunicipio,IdcTipoFaturamento,IdcClasseConsumidor,IdcSubclasse,IdcSubgrupoTarifario,IdcTipoCompraEnergia,NomCliente,NumCPFCNPJCliente,DscTipoSubsidio,VlrSubsidio
20582329,18/08/2023,12/2022,COPEL-DIS,4368898000106,4122206,1,1,3.2,11,1,MARIA ROSA FELIX DA CRUZ,***.577.990-**,SubsBaixaRenda,27.35
20582384,18/08/2023,12/2022,COPEL-DIS,4368898000106,4120150,1,1,3.2,11,1,IVONETE DE LARA,***.077.398-**,SubsBaixaRenda,16.13
20582527,18/08/2023,12/2022,COPEL-DIS,4368898000106,4119905,1,1,3.2,11,1,ANA ROSA SANTOS,***.969.392-**,SubsBaixaRenda,16.68
20582545,18/08/2023,12/2022,COPEL-DIS,4368898000106,4113700,1,1,3.5,11,1,AMANDA CAROLINE DE OLIVEIRA TERINATO,***.345.913-**,SubsBaixaRenda,24.91
20582635,18/08/2023,12/2022,COPEL-DIS,4368898000106,4113700,1,1,3.2,11,1,JANILSON DOMINGUES PAES SANTOS,***.966.597-**,SubsBaixaRenda,8.96


In [99]:
BaixaRenda2=BaixaRenda2.groupby(['AnmReferencia', 'CodIbgeMunicipio', 'IdcTipoFaturamento',
       'DscTipoSubsidio'])['VlrSubsidio'].agg(['sum','count']).reset_index()

In [100]:
BaixaRenda2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5362 entries, 0 to 5361
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AnmReferencia       5362 non-null   object 
 1   CodIbgeMunicipio    5362 non-null   int64  
 2   IdcTipoFaturamento  5362 non-null   int64  
 3   DscTipoSubsidio     5362 non-null   object 
 4   sum                 5362 non-null   float64
 5   count               5362 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 251.5+ KB


In [101]:
BaixaRenda2.sort_values(by='count',ascending=False)[:10]

,AnmReferencia,CodIbgeMunicipio,IdcTipoFaturamento,DscTipoSubsidio,sum,count
3831,12/2022,3550308,1,SubsBaixaRenda,31138920.14,625728
3244,12/2022,3304557,1,SubsBaixaRenda,9998058.05,364615
951,12/2022,2304400,1,SubsBaixaRenda,7742642.35,252580
171,12/2022,1501402,1,SubsBaixaRenda,7485269.70,188962
2164,12/2022,2927408,1,SubsBaixaRenda,4438354.43,150679
637,12/2022,2111300,1,SubsBaixaRenda,3871092.41,133957
3226,12/2022,3303500,1,SubsBaixaRenda,2873208.93,100339
1598,12/2022,2611606,1,SubsBaixaRenda,2981753.86,97294
884,12/2022,2211001,1,SubsBaixaRenda,2823573.91,90926
1698,12/2022,2704302,1,SubsBaixaRenda,2517482.90,80438


In [102]:
BaixaRenda2.to_csv('BaixaRenda2.csv')

In [103]:
url='https://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv'
porte=pd.read_csv(url, sep=";", encoding='Latin-1')

In [123]:
porte=porte[['IBGE',	'IBGE7',	'UF',	'Município',	'Região',	'População 2010',	'Porte']]
porte

,IBGE,IBGE7,UF,Município,Região,População 2010,Porte
0,110001,1100015,RO,Alta Floresta D´oeste,Região Norte,24392.0,Pequeno II
1,110002,1100023,RO,Ariquemes,Região Norte,90353.0,Médio
2,110003,1100031,RO,Cabixi,Região Norte,6313.0,Pequeno I
3,110004,1100049,RO,Cacoal,Região Norte,78574.0,Médio
4,110005,1100056,RO,Cerejeiras,Região Norte,17029.0,Pequeno I
...,...,...,...,...,...,...,...
5565,522200,5222005,GO,Vianópolis,Região Centro-Oeste,12548.0,Pequeno I
5566,522205,5222054,GO,Vicentinópolis,Região Centro-Oeste,7371.0,Pequeno I
5567,522220,5222203,GO,Vila Boa,Região Centro-Oeste,4735.0,Pequeno I
5568,522230,5222302,GO,Vila Propício,Região Centro-Oeste,5145.0,Pequeno I


In [133]:
BaixaRenda_municipio = pd.merge(BaixaRenda2, porte, how = 'left', right_on='IBGE7',left_on='CodIbgeMunicipio')

In [134]:
BaixaRenda_municipio.columns

Index(['AnmReferencia', 'CodIbgeMunicipio', 'IdcTipoFaturamento',
       'DscTipoSubsidio', 'sum', 'count', 'IBGE', 'IBGE7', 'UF', 'Município',
       'Região', 'População 2010', 'Porte'],
      dtype='object')

In [135]:
BaixaRenda_municipio=BaixaRenda_municipio[['AnmReferencia', 'DscTipoSubsidio', 'sum', 'count', 'IBGE', 'IBGE7', 'UF', 'Município', 'Porte']]

In [136]:
BaixaRenda_municipio=BaixaRenda_municipio.rename(columns={'sum':'Total_Subsidio_12_2022', 'count':'Total_consumidores','IBGE':'ibge_6','IBGE7':'ibge_7',
                                                         'Município':'municipio', 'Porte':'porte','AnmReferencia':'competencia'})



In [137]:
BaixaRenda_municipio

,competencia,DscTipoSubsidio,Total_Subsidio_12_2022,Total_consumidores,ibge_6,ibge_7,UF,municipio,porte
0,12/2022,SubsBaixaRenda,1868.73,67,NaN,NaN,NaN,NaN,NaN
1,12/2022,SubsBaixaRenda,49.00,2,NaN,NaN,NaN,NaN,NaN
2,12/2022,SubsBaixaRenda,27677.28,1056,110001.0,1100015.0,RO,Alta Floresta D´oeste,Pequeno II
3,12/2022,SubsBaixaRenda,172013.56,6275,110002.0,1100023.0,RO,Ariquemes,Médio
4,12/2022,SubsBaixaRenda,6287.68,241,110003.0,1100031.0,RO,Cabixi,Pequeno I
...,...,...,...,...,...,...,...,...,...
5357,12/2022,SubsBaixaRenda,13679.88,516,522205.0,5222054.0,GO,Vicentinópolis,Pequeno I
5358,12/2022,SubsBaixaRenda,8079.59,325,522220.0,5222203.0,GO,Vila Boa,Pequeno I
5359,12/2022,SubsBaixaRenda,7433.35,312,522230.0,5222302.0,GO,Vila Propício,Pequeno I
5360,12/2022,SubsBaixaRenda,2076173.90,72627,530010.0,5300108.0,DF,Brasília,Metrópole


In [138]:
BaixaRenda_municipio[(BaixaRenda_municipio['ibge_6'].isna())]

,competencia,DscTipoSubsidio,Total_Subsidio_12_2022,Total_consumidores,ibge_6,ibge_7,UF,municipio,porte
0,12/2022,SubsBaixaRenda,1868.73,67,NaN,NaN,NaN,NaN,NaN
1,12/2022,SubsBaixaRenda,49.00,2,NaN,NaN,NaN,NaN,NaN
5361,12/2022,SubsBaixaRenda,2048.64,78,NaN,NaN,NaN,NaN,NaN


In [139]:
BaixaRenda_municipio=BaixaRenda_municipio[(BaixaRenda_municipio['ibge_6'].notna())]

In [140]:
BaixaRenda_municipio.sort_values(by='Total_consumidores',ascending=False)[0:10]


,competencia,DscTipoSubsidio,Total_Subsidio_12_2022,Total_consumidores,ibge_6,ibge_7,UF,municipio,porte
3831,12/2022,SubsBaixaRenda,31138920.14,625728,355030.0,3550308.0,SP,São Paulo,Metrópole
3244,12/2022,SubsBaixaRenda,9998058.05,364615,330455.0,3304557.0,RJ,Rio de Janeiro,Metrópole
951,12/2022,SubsBaixaRenda,7742642.35,252580,230440.0,2304400.0,CE,Fortaleza,Metrópole
171,12/2022,SubsBaixaRenda,7485269.70,188962,150140.0,1501402.0,PA,Belém,Metrópole
2164,12/2022,SubsBaixaRenda,4438354.43,150679,292740.0,2927408.0,BA,Salvador,Metrópole
637,12/2022,SubsBaixaRenda,3871092.41,133957,211130.0,2111300.0,MA,São Luís,Metrópole
3226,12/2022,SubsBaixaRenda,2873208.93,100339,330350.0,3303500.0,RJ,Nova Iguaçu,Grande
1598,12/2022,SubsBaixaRenda,2981753.86,97294,261160.0,2611606.0,PE,Recife,Metrópole
884,12/2022,SubsBaixaRenda,2823573.91,90926,221100.0,2211001.0,PI,Teresina,Grande
1698,12/2022,SubsBaixaRenda,2517482.90,80438,270430.0,2704302.0,AL,Maceió,Metrópole


In [141]:
BaixaRenda_municipio.sort_values(by='Total_consumidores',ascending=True)[0:10]

,competencia,DscTipoSubsidio,Total_Subsidio_12_2022,Total_consumidores,ibge_6,ibge_7,UF,municipio,porte
4380,12/2022,SubsBaixaRenda,22.13,1,421720.0,4217204.0,SC,São Miguel do Oeste,Pequeno II
4330,12/2022,SubsBaixaRenda,0.59,1,420490.0,4204905.0,SC,Descanso,Pequeno I
4325,12/2022,SubsBaixaRenda,5.27,1,420290.0,4202909.0,SC,Brusque,Grande
4355,12/2022,SubsBaixaRenda,9.21,1,421090.0,4210902.0,SC,Modelo,Pequeno I
4379,12/2022,SubsBaixaRenda,24.24,1,421715.0,4217154.0,SC,São Miguel da Boa Vista,Pequeno I
4375,12/2022,SubsBaixaRenda,26.33,1,421600.0,4216008.0,SC,São Carlos,Pequeno I
4369,12/2022,SubsBaixaRenda,18.72,1,421430.0,4214300.0,SC,Rancho Queimado,Pequeno I
4328,12/2022,SubsBaixaRenda,23.79,1,420470.0,4204707.0,SC,Cunha Porã,Pequeno I
4334,12/2022,SubsBaixaRenda,1.18,2,420540.0,4205407.0,SC,Florianópolis,Grande
4313,12/2022,SubsBaixaRenda,52.92,2,420055.0,4200556.0,SC,Águas Frias,Pequeno I


In [144]:
BaixaRenda_municipio.to_csv('BaixaRenda_municipio.csv',encoding = 'UTF-8')